<a href="https://colab.research.google.com/github/neiraRail/soluciones_maze/blob/main/Soluciones_a_un_Laberinto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Funciones de objetivo

In [12]:
def llegamos(estado):
  return estado == [6,4]

def estamosBloqueados(mapa, pos):
  return mapa[pos[0]][pos[1]] == 9


### Funciones de movimiento

In [4]:
def moverAbajo(pos):
  return [pos[0]+1, pos[1]]

def moverArriba(pos):
  return [pos[0]-1, pos[1]]

def moverDerecha(pos):
  return [pos[0], pos[1]+1]

def moverIzquierda(pos):
  return [pos[0], pos[1]-1]

### Funciones de percepción

In [5]:
def checkAbajo(mapa, pos):
  if not pos[0] + 1 == len(mapa):
    return mapa[pos[0]+1][pos[1]] == 0
  else:
    return False

def checkArriba(mapa, pos):
  if not pos[0] == 0:
    return mapa[pos[0]-1][pos[1]] == 0
  else:
    return False
  
def checkDerecha(mapa, pos):
  if not pos[1] + 1 == len(mapa[0]): 
    return mapa[pos[0]][pos[1]+1] == 0
  else:
    return False
  
def checkIzquierda(mapa, pos):
  if not pos[1] == 0:
    return mapa[pos[0]][pos[1]-1] == 0 
  else:
    return False

def numeroDeCaminos(mapa, pos):
  return checkArriba(mapa, pos)+checkAbajo(mapa, pos)+checkDerecha(mapa, pos)+checkIzquierda(mapa, pos)

### Historial

In [6]:
def guardarMovimiento(historial, pos):
  historial.append(pos)
  return historial

### Funciones que modifican el entorno

In [23]:
def marcarBloqueo(mapa, pos):
  mapa[pos[0]][pos[1]] = 9
  return mapa

def marcarPos(mapa, pos):
  mapa[pos[0]][pos[1]] = 2
  return mapa

def imprimir(mapa):
  for linea in mapa:
    strlinea = ""
    for celda in linea:
      if celda == 1:
        strlinea+="█ "
      else:
        strlinea+=str(celda)+' '
    print(strlinea)

def limpiarRuta(mapa):
  for i in range(len(mapa)):
    for j in range(len(mapa[i])):
      if mapa[i][j] == 2:
        mapa[i][j] = 0
  return mapa


## Algoritmo principal:

- El agente se mueve siguiendo la siguiente preferencia: 
  - primero abajo
  - derecha
  - arriba
  - izquierda
- Si se queda estancado, marca el ultimo paso como bloqueado
- Si llega al objetivo se guarda la ruta que realizó como una solución
- Se repite hasta estar bloqueado o encontrar la solución

In [31]:
import copy

def encontrarSolucion(a, posInicial):
  mapa = copy.deepcopy(a)
  solucionado = False
  while not solucionado:
    pos = posInicial
    ruta = []
    seguirRuta = True
    while seguirRuta:
      if llegamos(pos):
        seguirRuta = False
        solucionado = True
        imprimir(mapa)
        mapa = limpiarRuta(mapa)
      elif estamosBloqueados(mapa, pos):
        seguirRuta = False
        solucionado = False
        imprimir(mapa)
        mapa = limpiarRuta(mapa)
        raise
      else:
        mapa = marcarPos(mapa, pos)
        ruta = guardarMovimiento(ruta, pos)
        if checkAbajo(mapa, pos):
          pos = moverAbajo(pos)
        elif checkDerecha(mapa, pos):
          pos = moverDerecha(pos)
        elif checkArriba(mapa, pos):
          pos = moverArriba(pos)
        elif checkIzquierda(mapa, pos):
          pos = moverIzquierda(pos)
        else:
          mapa = marcarBloqueo(mapa, pos)
          seguirRuta = False
          mapa = limpiarRuta(mapa)   

  
  
  print(ruta)
  return ruta, mapa

In [9]:
def difurcaciones(mapa, sol):
  dif = []
  for index, paso in enumerate(sol):
    if index == 0:
      if numeroDeCaminos(mapa, paso)>1:
        dif.append({
            "paso": paso,
            "sig":  sol[index+1]
        })
    elif numeroDeCaminos(mapa, paso) > 2:
      dif.append({
          "paso": paso,
          "sig":  sol[index+1]
      })

  return dif


def truncarRuta(ruta, pos):
  nueva = []
  for paso in ruta:
    if paso == pos:
      return nueva
    else:
      nueva.append(paso)

  return nueva

In [29]:
laberinto = [
    [0, 0, 0, 0, 0],
    [0, 1, 0, 1, 0],
    [0, 0, 0, 0, 0],
    [1, 0, 1, 0, 1],
    [1, 0, 1, 0, 0],
    [1, 0, 1, 1, 0],
    [1, 0, 1, 1, 0]
]

In [32]:
soluciones = []
try:
  sol, mapa = encontrarSolucion(laberinto, [0,0])
  soluciones.append(sol)
except:
  print("Sin solución")
else:
  difs = difurcaciones(mapa, sol)
  difs.reverse()
  print("-----$$$ Difurcaciones de la ruta principal")
  for dif in difs:
    iniciales = []
    localMapa = copy.deepcopy(mapa)
    ##Marcar ruta hasta ahora
    for paso in truncarRuta(sol, dif["paso"]):
      localMapa = marcarBloqueo(localMapa, paso)
    localMapa = marcarBloqueo(localMapa, dif['paso'])
    ##Intentar con todas las direcciones menos la siguiente de la primera solución
    if checkAbajo(localMapa, dif['paso']) and not moverAbajo(dif['paso']) == dif['sig']:
      iniciales.append(moverAbajo(dif['paso']))
    if checkDerecha(localMapa, dif['paso']) and not moverDerecha(dif['paso']) == dif['sig']:
      iniciales.append(moverDerecha(dif['paso']))
    if checkArriba(localMapa, dif['paso']) and not moverArriba(dif['paso']) == dif['sig']:
      iniciales.append(moverArriba(dif['paso']))
    if checkIzquierda(localMapa, dif['paso']) and not moverIzquierda(dif['paso']) == dif['sig']:
      iniciales.append(moverIzquierda(dif['paso']))
    #Repetir algoritmo de busqueda
    for inicial in iniciales:
      try:
        nuevaSol, otroMapa = encontrarSolucion(localMapa, inicial)
      except:
        print("Difurcasión sin solución")
      else:
        soluciones.append(nuevaSol)


print("------$$$ Soluciones")
print("nro de soluciones: "+str(len(soluciones)))

2 0 0 0 0 
2 █ 0 █ 0 
2 2 2 2 0 
█ 9 █ 2 █ 
█ 9 █ 2 2 
█ 9 █ █ 2 
█ 9 █ █ 0 
[[0, 0], [1, 0], [2, 0], [2, 1], [2, 2], [2, 3], [3, 3], [4, 3], [4, 4], [5, 4]]
-----$$$ Difurcaciones de la ruta principal
9 9 9 9 9 
9 █ 9 █ 9 
9 9 9 9 9 
█ 9 █ 0 █ 
█ 9 █ 0 0 
█ 9 █ █ 0 
█ 9 █ █ 0 
Difurcasión sin solución
9 0 2 2 2 
9 █ 2 █ 2 
9 9 9 2 2 
█ 9 █ 2 █ 
█ 9 █ 2 2 
█ 9 █ █ 2 
█ 9 █ █ 0 
[[1, 2], [0, 2], [0, 3], [0, 4], [1, 4], [2, 4], [2, 3], [3, 3], [4, 3], [4, 4], [5, 4]]
9 2 2 0 0 
0 █ 2 █ 0 
0 0 2 2 0 
█ 9 █ 2 █ 
█ 9 █ 2 2 
█ 9 █ █ 2 
█ 9 █ █ 0 
[[0, 1], [0, 2], [1, 2], [2, 2], [2, 3], [3, 3], [4, 3], [4, 4], [5, 4]]
------$$$ Soluciones
nro de soluciones: 3
